In [ ]:
import os
import math
import random
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
inception_v3_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=(299, 299, 3),
    pooling=None,
    classes=1000,
    classifier_activation='linear'
)


In [ ]:
for layer in inception_v3_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(inception_v3_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense (33, activation='linear')(x)


model = tf.keras.Model(inception_v3_model.input, x)

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [ ]:
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

In [ ]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
names = ['ა','ბ','გ','დ', 'ე','ვ', 'ზ', 'თ','ი','კ', 'ლ','მ','ნ','ო','პ','ჟ', 'რ', 'ს','ტ','უ', 'ფ','ქ', 'ღ', 'ყ','შ','ჩ', 'ც', 'ძ','წ','ჭ','ხ','ჯ','ჰ']

train_batches = train_gen.flow_from_directory(
    "./train",
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=8,
    shuffle=True,
    color_mode="rgb",
    classes=names   
)

val_batches = valid_gen.flow_from_directory(
    "./validation",
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=8,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

test_batches = test_gen.flow_from_directory(
    "./test",
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=8,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

In [ ]:
epochs = 15

model.fit(train_batches, validation_data=val_batches,
          epochs=epochs, verbose=2)

In [ ]:
model.evaluate(test_batches, verbose=2)